# QuickStart

## Working with data

```
PyTorch has two primitives to work with data: torch.utils.data.DataLoader and torch.utils.data.Dataset. Dataset stores the samples and their corresponding labels, and DataLoader wraps an iterable around the Dataset.

```

```
PyTorch offers domain-specific libraries such as TorchText, TorchVision, and TorchAudio, all of which include datasets. For this tutorial, we will be using a TorchVision dataset.
```

In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

```
The torchvision.datasets module contains Dataset objects for many real-world vision data like CIFAR, COCO (full list here). In this tutorial, we use the FashionMNIST dataset. Every TorchVision Dataset includes two arguments: transform and target_transform to modify the samples and labels respectively.
```

In [5]:
# Downloading Training data from open datasets.

training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

# Download the test data from open datasets.

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26.4M/26.4M [00:02<00:00, 12.5MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 209kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.99MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 20.7MB/s]


```
We pass the Dataset as an argument to DataLoader. This wraps an iterable over our dataset, and supports automatic batching, sampling, shuffling and multiprocess data loading. Here we define a batch size of 64, i.e. each element in the dataloader iterable will return a batch of 64 features and labels.
```

In [6]:
batch_size = 32

# Create dataloaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W] : {X.shape}")
    print(f"Shape of y: {y.shape}, {y.dtype}")
    break

Shape of X [N, C, H, W] : torch.Size([32, 1, 28, 28])
Shape of y: torch.Size([32]), torch.int64


## Creating Models

In [19]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"

print(f"Using {device} device")

# Define Model

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512,10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)
        

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Optimizing the Model Parameters

```
To train a model, we need a loss function and an optimizer.
```

In [20]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

```
In a single training loop, the model makes predictions on the training dataset (fed to it in batches), and backpropagates the prediction error to adjust the model’s parameters.
```

In [21]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()

    for batch, (X,y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute Prediction Error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch%100 == 0:
            loss, current = loss.item(), (batch+1) * len(X)
            print(f"loss: {loss:>7f}, [{current:5d}/{size:>5d}]")

```
We also check the model’s performance against the test dataset to ensure it is learning.
```

In [22]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)

    model.eval()

    test_loss, correct = 0,0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss/= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [23]:
epochs = 5

for t in range(epochs):
    print(f"Epochs {t+1}\n-------------------------------->")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epochs 1
-------------------------------->
loss: 2.304002, [   32/60000]
loss: 2.301297, [ 3232/60000]
loss: 2.271713, [ 6432/60000]
loss: 2.264773, [ 9632/60000]
loss: 2.243869, [12832/60000]
loss: 2.239437, [16032/60000]
loss: 2.220498, [19232/60000]
loss: 2.199400, [22432/60000]
loss: 2.189997, [25632/60000]
loss: 2.143776, [28832/60000]
loss: 2.109569, [32032/60000]
loss: 2.083447, [35232/60000]
loss: 2.173095, [38432/60000]
loss: 2.070304, [41632/60000]
loss: 2.031654, [44832/60000]
loss: 2.014212, [48032/60000]
loss: 2.060894, [51232/60000]
loss: 1.979140, [54432/60000]
loss: 1.890902, [57632/60000]
Test Error: 
 Accuracy: 59.2%, Avg loss: 1.900371 

Epochs 2
-------------------------------->
loss: 1.886398, [   32/60000]
loss: 1.908138, [ 3232/60000]
loss: 1.828564, [ 6432/60000]
loss: 1.787229, [ 9632/60000]
loss: 1.723834, [12832/60000]
loss: 1.772025, [16032/60000]
loss: 1.589005, [19232/60000]
loss: 1.581344, [22432/60000]
loss: 1.599913, [25632/60000]
loss: 1.538652, [28832

## Saving Models

In [24]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


## Loading Models

In [26]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only = True))

<All keys matched successfully>

In [27]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


# Expermenting by Changing ReLu to ReLu6

In [46]:
batch_size = 32

# Create dataloaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W] : {X.shape}")
    print(f"Shape of y: {y.shape}, {y.dtype}")
    break

Shape of X [N, C, H, W] : torch.Size([32, 1, 28, 28])
Shape of y: torch.Size([32]), torch.int64


In [47]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"

print(f"Using {device} device")

# Define Model

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU6(),
            nn.Linear(512, 512),
            nn.ReLU6(),
            nn.Linear(512,10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)
        

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU6()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU6()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [48]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

In [49]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()

    for batch, (X,y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute Prediction Error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch%100 == 0:
            loss, current = loss.item(), (batch+1) * len(X)
            print(f"loss: {loss:>7f}, [{current:5d}/{size:>5d}]")

In [50]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)

    model.eval()

    test_loss, correct = 0,0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss/= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [51]:
epochs = 35

for t in range(epochs):
    print(f"Epochs {t+1}\n-------------------------------->")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epochs 1
-------------------------------->
loss: 2.304760, [   32/60000]
loss: 2.305626, [ 3232/60000]
loss: 2.285548, [ 6432/60000]
loss: 2.267972, [ 9632/60000]
loss: 2.255940, [12832/60000]
loss: 2.238918, [16032/60000]
loss: 2.210743, [19232/60000]
loss: 2.185286, [22432/60000]
loss: 2.199803, [25632/60000]
loss: 2.133728, [28832/60000]
loss: 2.104254, [32032/60000]
loss: 2.084268, [35232/60000]
loss: 2.177850, [38432/60000]
loss: 2.066704, [41632/60000]
loss: 2.029638, [44832/60000]
loss: 2.020277, [48032/60000]
loss: 2.060759, [51232/60000]
loss: 1.972280, [54432/60000]
loss: 1.891477, [57632/60000]
Test Error: 
 Accuracy: 50.2%, Avg loss: 1.895700 

Epochs 2
-------------------------------->
loss: 1.878716, [   32/60000]
loss: 1.915860, [ 3232/60000]
loss: 1.835957, [ 6432/60000]
loss: 1.789571, [ 9632/60000]
loss: 1.733355, [12832/60000]
loss: 1.771088, [16032/60000]
loss: 1.608865, [19232/60000]
loss: 1.585099, [22432/60000]
loss: 1.603932, [25632/60000]
loss: 1.538150, [28832